In [2]:
!pip install assemblyai

   ---------------------------------------- 0.0/44.2 kB ? eta -:--:--
   ---------------------------------------- 44.2/44.2 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/163.3 kB ? eta -:--:--
   ---------------------------------------- 163.3/163.3 kB 4.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\Sarani\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import assemblyai as aai
aai.settings.api_key = "065e13309e1e4057b820b51d246d3a32"

In [7]:
# Or use a publicly-accessible URL:
audio_file = (
    "samples/test.wav"
)

In [11]:
def milliseconds_to_minutes(milliseconds):
    return milliseconds / (1000 * 60)

config = aai.TranscriptionConfig(
    speaker_labels=True,
)

transcript = aai.Transcriber().transcribe(audio_file, config)

for utterance in transcript.utterances:
    start_minutes = milliseconds_to_minutes(utterance.start)
    end_minutes = milliseconds_to_minutes(utterance.end)
    print(f"{start_minutes:.2f}")
    print(f"Speaker {utterance.speaker}: {utterance.text}")
    print(f"{end_minutes:.2f}")
    print()

0.08
Speaker A: My gosh. You've already produced a PowerPoint presentation.
0.13

0.13
Speaker B: I think it's already on actually. God, how do you make this thing work?
0.28

0.44
Speaker C: You have to.
0.46

0.51
Speaker B: Hello.
0.52

0.53
Speaker C: Have you done that?
0.55

0.55
Speaker B: I've plugged it in the back, but. Okay. Right. Okay. Right. Well, this is the kickoff meeting for our project and this is just what we're going to be doing over the next 25 minutes. So first of all, just to kind of make sure that we all know each other. I'm Laura and I'm the project manager.
1.25

1.25
Speaker A: Great.
1.26

1.26
Speaker B: Do you want to introduce yourself again?
1.29

1.29
Speaker D: Hi, I'm David and I'm supposed to be an industrial designer.
1.35

1.35
Speaker B: Okay.
1.36

1.36
Speaker A: And I'm Andrew and I'm a marketing expert.
1.45

1.45
Speaker E: I'm Greg and I'm User interface.
1.48

1.49
Speaker B: Great. Okay. So we're designing a new remote control. And. Oh, I